## Submitted to DR. Mustafa A. Elattar
##  TEAM NAMES & IDs:
> *   Abdelrhman Salah Salem, 211001698
> *   Abdelrahman Yasser Hussien, 211000008
> *    Youssef Ashraf ElNaggar, 211001713
> *   Youssef Hatem Mostafa, 211001689
> *   Youssef Mahmoud AbdelSamea, 211001545

##   Our Purpose:
### The main purpose of our analysis to highlight the biggest event in the middle east the last 3 months, which is isreal palestine war in addition to produce meaningful outputs of opinions on the IsrealPalestine subreddit posts and comments.

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/reddit-on-israel-palestine-daily-updated/reddit_opinion_PSE_ISR.csv
/kaggle/input/reddit-on-israel-palestine-daily-updated/legacy/pse_isr_reddit_comments.csv


In [2]:
import pandas as pd
import numpy as np
import random
import pytz
import datetime
from collections import defaultdict
from textblob import TextBlob
from wordcloud import WordCloud
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import re
import numpy as np
from bs4 import BeautifulSoup
from nltk.stem import WordNetLemmatizer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import seaborn as sns
import matplotlib.pyplot as plt
from wordcloud import WordCloud, STOPWORDS
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation

# Visualization
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib.dates import DateFormatter
import plotly.express as px
import plotly.graph_objects as go
import missingno as msno
from IPython.display import HTML,display

import warnings
warnings.filterwarnings("ignore")

/opt/conda/lib/python3.10/site-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [4]:
pd.set_option('display.max_columns', 25)
pd.set_option('display.max_rows', 70)
pd.set_option('display.min_rows', 5)
pd.set_option('display.max_colwidth', 1000)

In [5]:
df_original = pd.read_csv("/kaggle/input/reddit-on-israel-palestine-daily-updated/reddit_opinion_PSE_ISR.csv")

df = df_original.copy()
df.sample(5)

,comment_id,score,self_text,subreddit,created_time,post_id,author_name,controversiality,ups,downs,user_is_verified,user_account_created_time,user_awardee_karma,user_awarder_karma,user_link_karma,user_comment_karma,user_total_karma,post_score,post_self_text,post_title,post_upvote_ratio,post_thumbs_ups,post_total_awards_received,post_created_time
831221,k9893th,1,Yes my ass. You can check,IsraelPalestine,2023-11-14 16:22:27,17ugdi2,Historical_Issue_881,0,1,0,True,2023-11-09 03:00:14,0.0,0.0,1.0,-41.0,-40.0,0,"I know that Israel has done terrible things and they should write there wrongs, but I am so confused what has happened between October 7 to now. I don’t support either side, but I do remember when Hamas attacked in October. I’m pretty sure it was on a Jewish holiday, yet nobody really talked about that. Hamas kidnapped and cut the heads off of babies, but when Israel also kills children, only the IDF is at fault. Hamas hides in a hospital and Israel attacks, Israel is still in the wrong. I just feel like no matter what happens in this war, Israel will still be in the wrong.\n\nWhy do people only say that what Israel is bad, but defend Hamas when they do the exact same thing first and in a more cruel way?",Help me understand,0.50,0,0,2023-11-13 17:54:23
366814,kftfowf,41,Of all the predictions on this thread this is one of the few that I prefer the Gods of war absolutely ignore.,NonCredibleDefense,2024-01-01 10:08:35,18vk3x3,ButchersAssistant93,0,41,0,True,2022-12-10 21:23:05,0.0,0.0,1513.0,24954.0,26467.0,640,No election predictions this isn't /r/politics\n\nWanna make a noncredible prediction or post an imgur link to the 500th 2024 bingo sheet? Do it here. We will revisit this thread New Year's Eve next year and viciously attack how stupid you were this year.\n\nWho's plane is gonna mysteriously crash? Which 9zillion year old will die? What country will be the first to officially adopt the AeroGavin? It is all up to you!,2024 PREDICTIONS MEGATHREAD,0.99,640,0,2023-12-31 23:40:33
609440,kbvhfbp,1,"Even so but if you don’t think r/palestine is an echo chamber, you are truly delusional",IsraelPalestine,2023-12-03 22:21:16,189s8e9,deanmartinlawrence,0,1,0,True,2018-06-16 11:05:43,69.0,0.0,26556.0,31824.0,58449.0,172,"Sometimes I feel like jews are the only people who aren’t vehemently anti-Israel and don’t completely ignore/excuse 10/7, the hostage taking, and the disgusting human shield tactics by Hamas. Are there any non-Jews out there who disagree with and find themselves shocked by the rhetoric coming from far left?\n\nI honestly just don’t get it. Like I acknowledge that it is awful that so many Palestinians are dying, and I don’t have problems with people being critical of Israel’s response. But there’s a difference between doing that, and what I see so many people doing - pretending like 10/7 wasn’t a big deal, and that Hamas are just freedom fighters or downplaying their role in all of this? Like people have this mentality that Israel is just killing Palestinians because of nothing more than their race which is just utterly rediculous. People treat this as if they’re rooting for a sports team - denying or ignoring anything that doesn’t agree with their narrative. \n\nI just feel like I’...",Are there any non-Jews out there who aren’t staunchly anti-Israel,0.77,172,0,2023-12-03 12:08:50
925560,k7dw1am,28,"Did you miss the 7/10 event or are you just forcefully blind? Hamas has a wide range of weapons and are a real threat to any army.\n\nRocks are a threat against civilians surrounding and inside the West Bank, this is a whole different situation. Stop being idiotic.",CombatFootage,2023-11-01 16:45:13,17le7mu,Specialist_Glove3177,0,28,0,True,2020-10-25 21:03:33,0.0,0.0,668.0,2958.0,3626.0,1155,NaN,Additional footage released by IDF Spokesperson - 01.11.2023,0.90,1155,0,2023-11-01 15:06:51
518776,kd29ayj,0,Oy vey,AskMiddleEast,2023-12-12 17:32:27,18glc82,Odd_Responsibility94,1,0,0,False,NaN,0.0,0.0,0.0,0.0,0.0,339,NaN,"No

# **Data Exploration and Cleaning**

In [6]:
df.shape

(953902, 24)

In [7]:
df.columns

Index(['comment_id', 'score', 'self_text', 'subreddit', 'created_time',
       'post_id', 'author_name', 'controversiality', 'ups', 'downs',
       'user_is_verified', 'user_account_created_time', 'user_awardee_karma',
       'user_awarder_karma', 'user_link_karma', 'user_comment_karma',
       'user_total_karma', 'post_score', 'post_self_text', 'post_title',
       'post_upvote_ratio', 'post_thumbs_ups', 'post_total_awards_received',
       'post_created_time'],
      dtype='object')

In [8]:
df.head()

,comment_id,score,self_text,subreddit,created_time,post_id,author_name,controversiality,ups,downs,user_is_verified,user_account_created_time,user_awardee_karma,user_awarder_karma,user_link_karma,user_comment_karma,user_total_karma,post_score,post_self_text,post_title,post_upvote_ratio,post_thumbs_ups,post_total_awards_received,post_created_time
0,kt6xyrl,1,"No comments for 2 years and now, out of nowhere, you start spamming this on multiple posts? I'm guessing this is a bot account that was purchased.",PublicFreakout,2024-03-03 19:56:02,1b5nzpr,Krillinlt,0,1,0,True,2018-06-07 13:26:49,385.0,464.0,105.0,50705.0,51659.0,128,NaN,Zionist with Nail gun attempts to shoot pro-Palestine protestors,0.75,128,0,2024-03-03 18:24:53
1,kt6xwv8,1,So we're allowing footage from Palestine now?,CombatFootage,2024-03-03 19:55:43,1b5jltr,koufuki77,0,1,0,True,2014-02-01 07:20:46,48.0,250.0,1395.0,2682.0,4375.0,907,NaN,Israeli drone killing two hamas members,0.89,907,0,2024-03-03 15:20:29
2,kt6xwgs,1,It gets harder to watch every single day. These Zionist nazis will never face any consequences for their disgusting actions. Seeing all of this makes me wish I was religious so that I could look forward to these psychopaths rotting in hell. I desperately hope that there is some absolution.,Palestine,2024-03-03 19:55:39,1b5i808,Grapefruit__Witch,0,1,0,True,2024-02-21 03:48:55,0.0,0.0,1.0,2494.0,2495.0,452,NaN,"A military vehicle rams over two Palestinian teens riding an electric bike in the West Bank. Seconds later, an Israeli soldier throws down and shoots one of the boys point-blank.",0.98,452,0,2024-03-03 14:18:23
3,kt6xw3f,1,"Really? Examples please. When’s the last time the “world was up in arms?” Israel has enjoyed decades of international support from democratic nations and since its founding. If anything, its human rights violations have been glossed over. We all know why. It’s a dysfunctional yet symbiotic relationship to give the western world a dominant presence in the Mideast. You are right about one thing: the west only cares about Israel because it benefits them.",IsraelPalestine,2024-03-03 19:55:36,1b5gr54,333clh,0,1,0,True,2020-09-30 22:15:22,0.0,645.0,194.0,1003.0,1842.0,95,"Why do people care more about I/P than these? \n\nSyria - Bashar al-Assad killed at least 300,000 of his citizens and forced at least 4 million additional citizens to seek refuge outside Syria\n\nYemen - at least 150,000 were killed in Yemen because of a war started by the Houthis. (Some of these protestors even cheered when the Houthis attacked foreign ships.)\n\nSudan - at least 200,000 Sudanese have been killed in the conflict there\n\nChina - China has imprisoned a million Uyghurs and other Turkic Muslims into forced labor.\n\nIs it because Israel is Jewish?\n\nIs it because Jesus is from Israel?\n\nIs it because Jews are held to a higher standard than other peoples or religions?\n\nIs it because people think Israel is white people and think Palestinians are not?\n\nAny reasons at all? This is what’s on the news, this is what everyone’s talking about. I don’t see people asking Chinese people around the world about their fighting, but you see every Jew being asked about Israel. ...",Why do people care so much about this?,0.77,95,0,2024-03-03 13:06:30
4,kt6xw0n,1,+insulin,Palestine,2024-03-03 19:55:35,1b5my8i,_Snebb_,0,1,0,True,2020-01-25 11:04:09,520.0,0.0,7608.0,15417.0,23545.0,5,"Israel has banned water filtration systems, sleeping bags, women’s menstrual products and even cancer medication from entering Gaza.\n\nIt is astonishing at how low the occupation will publicly go.",Prohibited Items,0.78,5,0,2024-03-03 17:42:48


In [9]:
df.dtypes

comment_id                     object
score                           int64
self_text                      object
subreddit                      object
created_time                   object
post_id                        object
author_name                    object
controversiality                int64
ups                             int64
downs                           int64
user_is_verified               object
user_account_created_time      object
user_awardee_karma            float64
user_awarder_karma            float64
user_link_karma               float64
user_comment_karma            float64
user_total_karma              float64
post_score                      int64
post_self_text                 object
post_title                     object
post_upvote_ratio             float64
post_thumbs_ups                 int64
post_total_awards_received      int64
post_created_time              object
dtype: object

### ***Changing wrong data types***

In [10]:
# convert to datetime using pd.to_datetime
df['created_time'] = pd.to_datetime(df['created_time'])
df['post_created_time'] = pd.to_datetime(df['post_created_time'])
df['user_account_created_time'] = pd.to_datetime(df['user_account_created_time'])

df.dtypes

comment_id                            object
score                                  int64
self_text                             object
subreddit                             object
created_time                  datetime64[ns]
post_id                               object
author_name                           object
controversiality                       int64
ups                                    int64
downs                                  int64
user_is_verified                      object
user_account_created_time     datetime64[ns]
user_awardee_karma                   float64
user_awarder_karma                   float64
user_link_karma                      float64
user_comment_karma                   float64
user_total_karma                     float64
post_score                             int64
post_self_text                        object
post_title                            object
post_upvote_ratio                    float64
post_thumbs_ups                        int64
post_total

### ***Cleaning text columns for better analysis***

In [11]:
import zipfile
import os

zip_file_path = '/usr/share/nltk_data/corpora/wordnet.zip'
destination_directory = '/usr/share/nltk_data/corpora/'

# Check if the specific file exists in the destination directory
file_to_check = 'index.adv'
file_path_to_check = os.path.join(destination_directory, 'wordnet', file_to_check)

if not os.path.exists(file_path_to_check):
    # Unzip the entire archive if the specific file doesn't exist
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extractall(destination_directory)
    print(f"Successfully unzipped {zip_file_path} to {destination_directory}")
else:
    print(f"File {file_to_check} already exists. No need to unzip the archive.")

Successfully unzipped /usr/share/nltk_data/corpora/wordnet.zip to /usr/share/nltk_data/corpora/


In [12]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from bs4 import BeautifulSoup
import re

# Initialize the lemmatizer and stopwords
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

def clean_text(text):
    if isinstance(text, str):  # Check if the input is a string
        # Remove URLs
        text = re.sub(r'http\S+', '', text)
        
        # Remove HTML tags
        soup = BeautifulSoup(text, 'html.parser')
        text = soup.get_text()
        
        # Remove special characters and lowercase the text
        text = re.sub(r'[^a-zA-Z0-9\s]', '', text).lower()
        
        # Tokenization
        tokens = word_tokenize(text)
        
        # Lemmatization and removing stopwords
        tokens = [lemmatizer.lemmatize(token) for token in tokens if token not in stop_words]
        
        # Join the tokens back into a sentence
        cleaned_text = ' '.join(tokens)
        
        return cleaned_text
    else:
        return ''  # Return an empty string for non-string values

    
    
    
    
# usage on 'self_text' column 
df['clean_text'] = df['self_text'].apply(clean_text)

In [13]:
# without Initialize the lemmatizer and stopwords {self_text} column
for i in range(15):
    print(df.self_text[i])

No comments for 2 years and now, out of nowhere, you start spamming this on multiple posts? I'm guessing this is a bot account that was purchased.
So we're allowing footage from Palestine now?
It gets harder to watch every single day. These Zionist nazis will never face any consequences for their disgusting actions. Seeing all of this makes me wish I was religious so that I could look forward to these psychopaths rotting in hell. I desperately hope that there is some absolution.
Really? Examples please. When’s the last time the “world was up in arms?” Israel has enjoyed decades of international support from democratic nations and since its founding. If anything, its human rights violations have been glossed over. We all know why. It’s a dysfunctional yet symbiotic relationship to give the western world a dominant presence in the Mideast. You are right about one thing: the west only cares about Israel because it benefits them.
+insulin
🇵🇸
I don't think the aftermath was thought through,

In [14]:
# with Initialize the lemmatizer and stopwords for self_text column
for i in range(15):
    print(df.clean_text[i])

comment 2 year nowhere start spamming multiple post im guessing bot account purchased
allowing footage palestine
get harder watch every single day zionist nazi never face consequence disgusting action seeing make wish religious could look forward psychopath rotting hell desperately hope absolution
really example please whens last time world arm israel enjoyed decade international support democratic nation since founding anything human right violation glossed know dysfunctional yet symbiotic relationship give western world dominant presence mideast right one thing west care israel benefit
insulin

dont think aftermath thought thats also placing blame victim instead oppressor havent taught better theyre peaceful face violence theyre violent face violence unfair advantage oppressive force run world side
prophet pbuh abdulmejid hit hard
thats look like double limp wrist thank
lmaaooo
clever comeback terrible dodge
torah plenty passage hebrew take land group people dont understand find argu

In [15]:
# usage on 'post_self_text' column 
df['clean_post_self_text'] = df['post_self_text'].apply(clean_text)

KeyboardInterrupt: 

In [ ]:
# without Initialize the lemmatizer and stopwords post_self_text column
for i in range(15):
    print(df.post_self_text[i])

In [ ]:
# with Initialize the lemmatizer and stopwords post_self_text column
for i in range(15):
    print(df.clean_post_self_text[i])

#### ***Select data(posts+comments) starting from '2023-10-07'***

In [ ]:
print('Len. of data before 2023-10-07:' ,len(df))
start_date = pd.to_datetime('2023-10-07')

# data(posts+comments) starting from '2023-10-07'
filtered_df = df[(df['post_created_time'] >= start_date) & (df['created_time'] >= start_date)]
print('Len. of data After 2023-10-07:',len(filtered_df))
print('Num. of dropped rows:',len(df)-len(filtered_df))

In [ ]:
# Get total number of missing values for all the dataset
filtered_df.isnull().sum().sum()

In [ ]:
filtered_df.isnull().sum()

***The post_self_text column contains the most null values,but This doesn't necessarily mean posts lack content; it's due to the dataset structure where post details get repeated for each comment.***

***See the dublicated data***

In [ ]:
# Check the dublicated posts in dataset
print(f" Duplicate Posts: {filtered_df[['post_title','post_self_text']].duplicated().sum()}")

In [ ]:
# Check the dublicated comments in dataset
print(f'Duplicate Comments: {filtered_df[["self_text"]].duplicated().sum()}')

> **However there are many duplicated values, but we can't remove all of them. Perhaps it will affect the results, due to many people can share the same post or have the same agreement in the comment.**

***show the most duplicated values***

In [ ]:
#show the most duplicated values in posts
df_duplicated_postss = filtered_df[filtered_df.duplicated(subset=['post_self_text'])]
post_self_text_countss = df_duplicated_postss['post_self_text'].value_counts().reset_index()
post_self_text_countss.columns = ['post_self_text', 'count']
post_self_text_countss.head(15)

In [ ]:
#show the most duplicated values in comments
df_duplicated_comments= filtered_df[filtered_df.duplicated(subset=['self_text'])]
self_text_counts = df_duplicated_comments['self_text'].value_counts().reset_index()
self_text_counts.columns = ['self_text', 'count']
self_text_counts.head(15)

# **sentiment analysis**


***Starting with posts sentiment analysis***

> Some reddit posts can only have a post title and a photo or video. This could be useful for identifying posts.

# 1. Posts Sentiment Analysis  

***i. sentiment analysis for post title***

In [ ]:
post_sentimenet_filtered_df=filtered_df[(~filtered_df['post_title'].isna())]
# Remove duplicate values from the 'post_title' column , as the one post may be duplicated as it has many comments
post_sentimenet_filtered_df = post_sentimenet_filtered_df.drop_duplicates(subset=['post_title'])

# Calc sentiment score for each record
post_sentimenet_filtered_df['post_sentiment'] = post_sentimenet_filtered_df['post_title'].apply(lambda x: TextBlob(str(x)).sentiment.polarity)
# Assign sentiment category based on sentiment score
post_sentimenet_filtered_df['post_sentiment_category'] = post_sentimenet_filtered_df['post_sentiment'].apply(lambda x: 'Positive' if x > 0 else ('Negative' if x < 0 else 'Neutral'))

# Count the occurrences of each sentiment category
sentiment_counts = post_sentimenet_filtered_df['post_sentiment_category'].value_counts()
sentiment_counts_df = sentiment_counts.reset_index()
sentiment_counts_df.columns = ['Post_Sentiment', 'Count']
sentiment_counts_df = sentiment_counts_df.sort_values(by='Count', ascending=False)
sentiment_counts_df.style.background_gradient(cmap='YlGnBu')

In [ ]:
fig = go.Figure(go.Funnelarea(
    text =sentiment_counts_df.Post_Sentiment,
    values = sentiment_counts_df.Count,
    title = {"position": "top center"}
    ))
fig.update_layout(
    title="Funnel-Chart of Sentiment Distribution | Posts",
    title_x=0.5, width=500, height=400)
fig.show()


plt.figure(figsize=(5, 5))
ax = plt.pie(x =post_sentimenet_filtered_df['post_sentiment_category'].value_counts() , labels=post_sentimenet_filtered_df['post_sentiment_category'].value_counts().index, autopct = '%1.1f%%', explode = [0.03, 0.03, 0.08])
plt.title('Sentiment Distribution | Posts')
plt.show()

***ii. sentiment analysis for post title***

In [ ]:
post_self_text_filtered_df = filtered_df[(~filtered_df['clean_post_self_text'].isna())]
# Remove duplicate values from the 'post_title' column , as the one post may be duplicated as it has many comments
post_self_text_filtered_df = post_self_text_filtered_df.drop_duplicates(subset=['post_title'])

# Calc sentiment score for each record
post_self_text_filtered_df['post_sentiment'] = post_self_text_filtered_df['clean_post_self_text'].apply(lambda x: TextBlob(str(x)).sentiment.polarity)
# Assign sentiment category based on sentiment score
post_self_text_filtered_df['post_sentiment_category'] = post_self_text_filtered_df['post_sentiment'].apply(lambda x: 'Positive' if x > 0 else ('Negative' if x < 0 else 'Neutral'))

# Count the occurrences of each sentiment category
sentiment_counts = post_self_text_filtered_df['post_sentiment_category'].value_counts()
sentiment_counts_df = sentiment_counts.reset_index()
sentiment_counts_df.columns = ['Post_Sentiment', 'Count']
sentiment_counts_df = sentiment_counts_df.sort_values(by='Count', ascending=False)
sentiment_counts_df.style.background_gradient(cmap='YlGnBu')

In [ ]:
fig = go.Figure(go.Funnelarea(
    text =sentiment_counts_df.Post_Sentiment,
    values = sentiment_counts_df.Count,
    title = {"position": "top center"}
    ))
fig.update_layout(
    title="Funnel-Chart of Sentiment Distribution | Posts",
    title_x=0.5, width=500, height=400)
fig.show()


plt.figure(figsize=(5, 5))
ax = plt.pie(x =post_self_text_filtered_df['post_sentiment_category'].value_counts() , labels=post_sentimenet_filtered_df['post_sentiment_category'].value_counts().index, autopct = '%1.1f%%', explode = [0.03, 0.03, 0.08])
plt.title('Sentiment Distribution | Posts')
plt.show()

 **Examples of of Positive, Negative, and Neutral posts**

In [ ]:
# Neutral posts
post_self_text_filtered_df[
    (post_self_text_filtered_df['post_sentiment_category'] == 'Neutral') &  (post_self_text_filtered_df['post_self_text'].notnull())
    ].sample(n=5, random_state=random.seed())[['post_title', 'post_self_text', 'post_sentiment', 'post_sentiment_category']]

In [ ]:
# positive posts
post_self_text_filtered_df[
    (post_self_text_filtered_df['post_sentiment_category'] == 'Positive') &  (post_self_text_filtered_df['post_self_text'].notnull())
    ].sample(n=5, random_state=random.seed())[['post_title', 'post_self_text', 'post_sentiment', 'post_sentiment_category']]

In [ ]:
# Negative posts
post_self_text_filtered_df[
    (post_self_text_filtered_df['post_sentiment_category'] == 'Negative') &  (post_self_text_filtered_df['post_self_text'].notnull())
    ].sample(n=5, random_state=random.seed())[['post_title', 'post_self_text', 'post_sentiment', 'post_sentiment_category']]

**Analyzing Sentiment Trends Over Time**

In [ ]:
# Create legend patches
positive_patch = mpatches.Patch(color='green', label='Positive')
negative_patch = mpatches.Patch(color='red', label='Negative')
neutral_patch = mpatches.Patch(color='blue', label='Neutral')

# Group by 'created_time' and calculate the mean sentiment for each time period
sentiment_by_time = post_self_text_filtered_df.groupby(post_self_text_filtered_df['post_created_time'].dt.date)['post_sentiment'].mean()

# Replace 'sentiment_scores' with the actual variable or column containing sentiment scores
sentiment_scores = post_self_text_filtered_df['post_sentiment']

# Calculate dynamic thresholds based on percentiles
percentile_25 = np.percentile(sentiment_scores, 25)
percentile_75 = np.percentile(sentiment_scores, 75)

# Classify sentiments based on dynamic thresholds
post_self_text_filtered_df['post_sentiment_category'] = np.where(
    post_self_text_filtered_df['post_sentiment'] < percentile_25, 'Negative',
    np.where( post_self_text_filtered_df['post_sentiment'] > percentile_75, 'Positive','Neutral' ))

# Plot the sentiment over time with lines connecting points and different colors for each category
plt.figure(figsize=(15, 6))

for idx, row in sentiment_by_time.reset_index().iterrows():
    date = row['post_created_time']
    sentiment = post_self_text_filtered_df.loc[post_self_text_filtered_df['post_created_time'].dt.date == date, 'post_sentiment_category'].iloc[0]

    # Assign colors based on sentiment category
    color = 'green' if sentiment == 'Positive' else ('red' if sentiment == 'Negative' else 'blue')

    # Plot the point
    plt.plot(date, row['post_sentiment'], marker='o', color=color)

    # Connect with lines to the previous point
    if idx > 0:
        plt.plot([previous_date, date], [previous_sentiment, row['post_sentiment']], color=color)

    previous_date = date
    previous_sentiment = row['post_sentiment']

# Add legend with colored boxes
plt.legend(handles=[positive_patch, negative_patch, neutral_patch], title='Sentiment Category')
plt.title('Sentiment Over Time | Posts')
plt.xlabel('Date')
plt.ylabel('Mean Sentiment Score')

# Display every other date on x-axis
plt.xticks(sentiment_by_time.index[::2], rotation=45, ha='right')

plt.show()

In [ ]:
# Filter posts by sentiment category
positive_posts = post_self_text_filtered_df[post_self_text_filtered_df['post_sentiment_category'] == 'Positive']
negative_posts = post_self_text_filtered_df[post_self_text_filtered_df['post_sentiment_category'] == 'Negative']
neutral_posts = post_self_text_filtered_df[post_self_text_filtered_df['post_sentiment_category'] == 'Neutral']

# Group by 'post_created_time' and calculate counts
positive_counts_per_day = positive_posts.groupby(positive_posts['post_created_time'].dt.date).size()
negative_counts_per_day = negative_posts.groupby(negative_posts['post_created_time'].dt.date).size()
neutral_counts_per_day = neutral_posts.groupby(neutral_posts['post_created_time'].dt.date).size()

# Plotting using Seaborn
plt.figure(figsize=(15, 6))

# Plot lines for each sentiment category
sns.lineplot(x=positive_counts_per_day.index, y=positive_counts_per_day.values, label='Positive',  color='green')
sns.lineplot(x=negative_counts_per_day.index, y=negative_counts_per_day.values, label='Negative',  color='red')
sns.lineplot(x=neutral_counts_per_day.index, y=neutral_counts_per_day.values, label='Neutral' , color='blue')

plt.title('Sentiment Posts Over Time')
plt.xlabel('Date')
plt.ylabel('Number of Posts')
plt.legend()
plt.xticks(positive_counts_per_day.index[::2], rotation=45, ha='right')  # Display every second date for better visibility
plt.show()

# **2. Comments sentiment analysis**

In [ ]:
sentiment_filtered_df=filtered_df[(~filtered_df['clean_text'].isna())]
sentiment_filtered_df = sentiment_filtered_df.drop_duplicates(subset=['clean_text'])

sentiment_filtered_df['comment_sentiment'] = sentiment_filtered_df['clean_text'].apply(lambda x: TextBlob(str(x)).sentiment.polarity)
sentiment_filtered_df['comment_sentiment_category'] = sentiment_filtered_df['comment_sentiment'].apply(lambda x: 'Positive' if x > 0 else ('Negative' if x < 0 else 'Neutral'))

# Count the occurrences of each sentiment category
sentiment_counts_df = pd.DataFrame(sentiment_filtered_df['comment_sentiment_category'].value_counts())
sentiment_counts_df.columns = ['Count']
sentiment_counts_df.index.name = 'comment_sentiment'
sentiment_counts_df.reset_index(inplace=True)
sentiment_counts_df = sentiment_counts_df.sort_values(by='Count', ascending=False)
sentiment_counts_df.style.background_gradient(cmap='YlGnBu')

In [ ]:
plt.figure(figsize=(5, 5))
ax = plt.pie(x =sentiment_filtered_df['comment_sentiment_category'].value_counts() , labels=sentiment_filtered_df['comment_sentiment_category'].value_counts().index, autopct = '%1.1f%%', explode = [0.03, 0.03, 0.08])
plt.title('Sentiment Distribution | Comments')
plt.show()

**Examples of Positive, Negative, and Neutral comments individually**

In [ ]:
# Neutral comments
sentiment_filtered_df[sentiment_filtered_df['comment_sentiment_category'] == 'Neutral'].sample(n=5, random_state=random.seed())[['self_text', 'comment_sentiment', 'comment_sentiment_category']]

In [ ]:
# Positive comments
sentiment_filtered_df[sentiment_filtered_df['comment_sentiment_category'] == 'Positive'].sample(n=5, random_state=random.seed())[['self_text', 'comment_sentiment', 'comment_sentiment_category']]

In [ ]:
# Negative comments
sentiment_filtered_df[sentiment_filtered_df['comment_sentiment_category'] == 'Negative'].sample(n=5, random_state=random.seed())[['self_text', 'comment_sentiment', 'comment_sentiment_category']]

**Analyzing comments Trends Over Time**

In [ ]:
# Group by 'created_time' and calculate the mean sentiment for each time period
sentiment_by_time = sentiment_filtered_df.groupby(sentiment_filtered_df['created_time'].dt.date)['comment_sentiment'].mean()

# Replace 'sentiment_scores' with the actual variable or column containing sentiment scores
sentiment_scores = sentiment_filtered_df['comment_sentiment']

# Calculate dynamic thresholds based on percentiles
percentile_25 = np.percentile(sentiment_scores, 25)
percentile_75 = np.percentile(sentiment_scores, 75)

# Classify sentiments based on dynamic thresholds
sentiment_filtered_df['comment_sentiment_category'] = np.where(
    sentiment_filtered_df['comment_sentiment'] < percentile_25, 'Negative',
    np.where(
        sentiment_filtered_df['comment_sentiment'] > percentile_75, 'Positive',
        'Neutral'
    )
)

# Plot the sentiment over time with lines connecting points and different colors for each category
plt.figure(figsize=(12, 6))

for idx, row in sentiment_by_time.reset_index().iterrows():
    date = row['created_time']
    sentiment = sentiment_filtered_df.loc[sentiment_filtered_df['created_time'].dt.date == date, 'comment_sentiment_category'].iloc[0]

    # Assign colors based on sentiment category
    color = 'green' if sentiment == 'Positive' else ('red' if sentiment == 'Negative' else 'blue')

    # Plot the point
    plt.plot(date, row['comment_sentiment'], marker='o', color=color)

    # Connect with lines to the previous point
    if idx > 0:
        plt.plot([previous_date, date], [previous_sentiment, row['comment_sentiment']], color=color)

    previous_date = date
    previous_sentiment = row['comment_sentiment']

# Add legend with colored boxes
plt.legend(handles=[positive_patch, negative_patch, neutral_patch], title='Sentiment Category')
plt.title('Sentiment Over Time | Comments')
plt.xlabel('Date')
plt.ylabel('Mean Sentiment Score')

# Display every other date on x-axis
plt.xticks(sentiment_by_time.index[::2], rotation=45, ha='right')

plt.show()

In [ ]:
# Filter comments by sentiment category
positive_comment = sentiment_filtered_df[sentiment_filtered_df['comment_sentiment_category'] == 'Positive']
negative_comment = sentiment_filtered_df[sentiment_filtered_df['comment_sentiment_category'] == 'Negative']
neutral_comment = sentiment_filtered_df[sentiment_filtered_df['comment_sentiment_category'] == 'Neutral']

# Group by 'post_created_time' and calculate counts
positive_counts_per_day = positive_comment.groupby(positive_comment['created_time'].dt.date).size()
negative_counts_per_day = negative_comment.groupby(negative_comment['created_time'].dt.date).size()
neutral_counts_per_day = neutral_comment.groupby(neutral_comment['created_time'].dt.date).size()

plt.figure(figsize=(15, 6))

# Plot lines for each sentiment category
sns.lineplot(x=positive_counts_per_day.index, y=positive_counts_per_day.values, label='Positive',  color='green')
sns.lineplot(x=negative_counts_per_day.index, y=negative_counts_per_day.values, label='Negative',  color='red')
sns.lineplot(x=neutral_counts_per_day.index, y=neutral_counts_per_day.values, label='Neutral' , color='blue')

plt.title('Sentiment Comments Over Time')
plt.xlabel('Date')
plt.ylabel('Number of Comments')
plt.legend()
plt.xticks(positive_counts_per_day.index[::2], rotation=45, ha='right')  # Display every second date for better visibility
plt.show()

# EDA for the Dataset


In [ ]:
filtered_df['post_created_time'] = pd.to_datetime(filtered_df['post_created_time'])

# Calculate date difference
date_difference = filtered_df['post_created_time'].max() - filtered_df['post_created_time'].min()

# Calculate years, months, and days
years = date_difference.days // 365
months = (date_difference.days % 365) // 30
days = (date_difference.days % 365) % 30

result = f"Data covers a period of {years} years, {months} months, {days} days, \nfrom {filtered_df['post_created_time'].min().date()} till {filtered_df['post_created_time'].max().date()}"
print(result)
print(filtered_df.shape)

## Highest post score & comment

In [ ]:
pd.DataFrame(filtered_df.loc[filtered_df['post_score'].idxmax()][[ 'post_title', 'post_score', 'subreddit', 'post_created_time']]).T

In [ ]:
pd.DataFrame(filtered_df.loc[filtered_df['score'].idxmax()][['self_text', 'post_title', 'score', 'subreddit', 'created_time']]).T

## Total Authors

In [ ]:
print("Number of unique authors:",  filtered_df['author_name'].nunique())


### Correlation Analysis between scores, karma, upvotes/downvotes, and user_total_karma to identify patterns.

In [ ]:
# Select numerical features for correlation analysis
numerical_columns = filtered_df.select_dtypes(include='number').columns

# Create a subset DataFrame with selected features
correlation_df = filtered_df[numerical_columns]

# Calculate correlation matrix
correlation_matrix = correlation_df.corr()


mask = np.triu(np.ones_like(correlation_matrix, dtype=bool))
plt.figure(figsize=(10, 5))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt='.2f', linewidths=.5, mask=mask)
plt.title('Correlation Matrix 12-1-2024')
plt.show()

🍉 Insights 🍉

From the provided correlation matrix, we can see that:

1. High Correlation (0.75)to(1.0):

* the columns "post_thumbs_ups" and "post_score" also have a perfect positive correlation (1.0), suggesting redundancy. We may choose to keep "post_score".
* Similarly but with a lower score, the columns "user_total_karma" and "user_comment_karma" also have a high positive correlation (0.82), suggesting redundancy. We choose to drop "user_comment_karma".
* In addition to another high positive correlation (0.75). we choose to drop "user_link_karma", and keep"user_total_karma".

2. medium Correlations:

* User_awardee_karma, user_total_karma have a moderate positive correlation, suggesting that users who write comments are also likely to takes awards.
3. Columns with lowest Correlation:

* The column "downs"has lowest correlation with other features, indicating no linear relationship. We consider dropping "downs" as it does not contribute meaningfully to our analysis

In [ ]:
filtered_df[['post_thumbs_ups','post_score','user_total_karma','user_comment_karma','user_link_karma','downs']]

### droping the unmeaningful features from the correlation analysis 

In [ ]:
filtered_df= filtered_df.drop(columns=['downs','post_thumbs_ups','user_comment_karma','user_link_karma'])
filtered_df.columns

In [ ]:
filtered_df.dtypes

# Descriptive Statisics

In [ ]:
# Descriptive statistics for numerical features
filtered_df.describe().T

In [ ]:
# Descriptive statistics for categorical features
filtered_df.describe(include='object').T

### exploring isrealpalestine Subreddit comments overtime

In [ ]:
popular_subreddits = filtered_df['subreddit'].value_counts()
popular_subreddits.columns=['subreddit', 'count']
popular_subreddits

In [ ]:
# Filter DataFrame for 'IsraelPalestine' subreddit
filtered_df_israel_palestine = filtered_df[filtered_df['subreddit'] == 'IsraelPalestine']

# Drop duplicates based on 'author_name' and 'user_account_created_time'
filtered_df_unique = filtered_df_israel_palestine.drop_duplicates(subset=['author_name', 'self_text'], keep='first')

filtered_df_unique['created_time'] = pd.to_datetime(filtered_df_unique['created_time']).dt.date

# Get the dates with the most account creations
top_dates = filtered_df_unique['created_time'].value_counts()

# Create a DataFrame from the top dates
top_dates_df = pd.DataFrame({'Date': top_dates.index, 'Account_Creations': top_dates.values})

# Sort the DataFrame by date
top_dates_df = top_dates_df.sort_values(by='Date')

plt.figure(figsize=(12, 6))
lineplot = sns.lineplot(x='Date', y='Account_Creations', data=top_dates_df, palette='viridis')

# Add labels to the data points
for x, y in zip(top_dates_df['Date'], top_dates_df['Account_Creations']):
    plt.text(x, y, str(y), ha='center', va='bottom', fontsize=10, rotation=45)

# Set the x-axis tick labels
date_ticks = top_dates_df['Date'][::2]  # Display every second date
plt.xticks(date_ticks, rotation=45, ha='right')

plt.title('Daily Commenting Activity | "IsraelPalestine" Subreddit ')
plt.xlabel('Date')
plt.ylabel('Account Creations')
plt.tight_layout()
plt.show()

## 🍉 Insights 🍉

The Earliest Comment Activity:
* Notably, "IsraelPalestine"  started in early November.

Top Day of highest comment rate:

* It's intriguing to observe that "Israelpalestine" subreddit experienced peaks in commenting around mid-November.
* "IsraelPalestine" stands out as having the highest total number of comments on its top day, reaching approximately +6359 comments on November 19, 2023.

Day with the Lowest Comment Rate:

* On November 6, 2023, the 'IsraelPalestine' subreddit experienced the lowest commenting rate, which is the first day for this Subreddit commuiny, with around 1063 comments.

Consistency and Variability:

* The data shows varying levels of commenting activity throughout the period, with fluctuations in comments creations. Yet a notable consistency is observed during mid-November, where the subreddit maintained a relatively high level of commenting activity over several consecutive days.

Decrease in Engagement:

* The gradual decline in comments towards the end of the dataset, especially from lately December, indicates a decrease in engagement.

### Exploring the Most Engaging Subreddit | IsraelPalestine

In [ ]:
# Count unique active users in the 'IsraelPalestine' subreddit
unique_users_count = filtered_df[filtered_df['subreddit'] == 'IsraelPalestine']['author_name'].nunique()
display(HTML(f"<h3>Number of unique users in IsraelPalestine subreddit: <b style='color:red'>{unique_users_count}</b></h3>"))

In [ ]:
# Filter posts related to 'IsraelPalestine' subreddit
posts_israel_palestine = filtered_df[(filtered_df['subreddit'] == 'IsraelPalestine') & (filtered_df['post_self_text'].notnull())]
# Drop duplicate posts
unique_posts = posts_israel_palestine.drop_duplicates(subset='post_self_text')
posts_count = len(unique_posts)
display(HTML(f"<h3>Number of unique posts in IsraelPalestine subreddit : <b style='color:red'>{posts_count}</b></h3>"))

In [ ]:
# Filter comments related to 'IsraelPalestine' subreddit
comments_israel_palestine = filtered_df[(filtered_df['subreddit'] == 'IsraelPalestine') & (filtered_df['self_text'].notnull())]
# Drop duplicate comments
unique_comments = comments_israel_palestine.drop_duplicates(subset='self_text')
# Get the counts
comments_count = len(unique_comments)
display(HTML(f"<h3>Number of unique comments in IsraelPalestine subreddit : <b style='color:red'>{comments_count}</b></h3>"))

In [ ]:
# Find the earliest post_created_time and corresponding post_self_text
earliest_post = posts_israel_palestine.loc[posts_israel_palestine['post_created_time'].idxmin()]
earliest_post_time = earliest_post['post_created_time']
earliest_post_text = earliest_post['post_self_text']
display(HTML(f"<h4>Earliest post created time : <b style='color:red'>{earliest_post_time}</b></h4>"))
display(HTML(f"<h4>Earliest post : <b style='color:red'>{earliest_post_text}</b></h4>"))
horizontal_line= '<hr style="border: none; background-color: #000000; height: 1px; margin: 5px 0;">'

display(HTML(horizontal_line))

last_post = posts_israel_palestine.loc[posts_israel_palestine['post_created_time'].idxmax()]
last_post_time = last_post['post_created_time']
last_post_text = last_post['post_self_text']
display(HTML(f"<h4>Last post created time : <b style='color:red'>{last_post_time}</b></h4>"))
display(HTML(f"<h4>Last post : <b style='color:red'>{last_post_text}</b></h4>"))

### WORDCLOUD OF COMMENTS

In [ ]:
from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt

# Assuming 'post_created_time' and 'post_self_text' are columns in israel_palestine_df
comment_words = ''
stopwords = set(STOPWORDS)

# Iterate through the DataFrame rows
for index, row in sentiment_filtered_df.iterrows():
    # Concatenate 'post_self_text' values
    val = str(row['self_text'])
    
    # Split the value into tokens
    tokens = val.split()

    # Convert each token to lowercase
    tokens = [token.lower() for token in tokens]

    # Join the lowercase tokens
    comment_words += " ".join(tokens) + " "

# Generate WordCloud
wordcloud = WordCloud(width=800, height=800,
                      background_color='white',
                      stopwords=stopwords,
                      min_font_size=10).generate(comment_words)

# Plot the WordCloud image
plt.figure(figsize=(8, 8), facecolor=None)
plt.imshow(wordcloud,  interpolation='bilinear')
plt.axis("off")
plt.tight_layout(pad=0)
plt.title("comments on 12-1-2024")
plt.show()

### Trends Period

In [ ]:
filtered_df_unique = filtered_df.copy()

filtered_df_unique['created_time'] = pd.to_datetime(filtered_df_unique['created_time']).dt.date

# Get the dates with the most account creations
top_dates = filtered_df_unique['created_time'].value_counts()

# Create a DataFrame from the top dates
top_dates_df = pd.DataFrame({'Date': top_dates.index, 'Comment_Creations': top_dates.values})

# Sort the DataFrame by date
top_dates_df = top_dates_df.sort_values(by='Date')

# Plotting using Seaborn
plt.figure(figsize=(20,6))
lineplot = sns.lineplot(x='Date', y='Comment_Creations', data=top_dates_df, palette='viridis')

# Add labels to the data points
for x, y in zip(top_dates_df['Date'], top_dates_df['Comment_Creations']):
    plt.text(x, y, str(y), ha='center', va='bottom', fontsize=10, rotation=45)

# Set the x-axis tick labels
date_ticks = top_dates_df['Date'][::2]  # Display every second date
plt.xticks(date_ticks, rotation=45, ha='right')

# Highlight the time period from 24 November 2023 to 30 November 2023 in a different color
highlight_start_date = pd.Timestamp('2023-11-24')
highlight_end_date = pd.Timestamp('2023-11-30')

lineplot.axvspan(highlight_start_date, highlight_end_date, color='lightcoral', alpha=0.3, label='Temporary Ceasefire Period')


plt.title('Temporal Trends - Comments per Day | All Subreddits')
plt.xlabel('Date')
plt.ylabel('Total Number of Comments')
plt.legend( title_fontsize='12', loc='upper left')
plt.tight_layout()
plt.show()

### Ceasefire Impact:

* There is a noticeable decline in the number of comments from November 24th to November 30th, possibly due to the announced ceasefire during that period. This decline aligns with a decrease in commenting activity.

In [ ]:
# Assuming 'post_created_time' is a datetime column in israel_palestine_df
filtered_df['post_created_time'] = pd.to_datetime(filtered_df['post_created_time'])

# Group by date and count the number of posts for each day
filtered_df.groupby(filtered_df['post_created_time'].dt.date)['post_self_text'].count()

# 🍉 Analysis

In [ ]:
# Assuming 'self_text' is the column containing posts/comments in your DataFrame
emoji = '🍉'
emoji_counts = filtered_df['self_text'].str.count(emoji)

# Create a new column in the DataFrame with the counts
filtered_df['emoji_counts'] = emoji_counts

# Sum the counts to get the total occurrences
total_occurrences = emoji_counts.sum()

# Print the total occurrences
print(f'Total occurrences of 🍉 emoji: {total_occurrences}')

# Plot a bar chart with color gradient to visualize the occurrences
plt.figure(figsize=(10, 6))
sns.scatterplot(x=filtered_df.index, y=filtered_df['emoji_counts'], color='red')
plt.xlabel('Index')
plt.ylabel('Emoji Counts')
plt.title('Scatter Plot of 🍉 Emoji Counts in Posts/Comments')
plt.show()

In [ ]:
# Assuming 'self_text' is the column containing posts/comments in your DataFrame
emoji = '🍉'
emoji_counts = filtered_df['self_text'].str.count(emoji)

# Create a new column in the DataFrame with the counts
filtered_df['emoji_counts'] = emoji_counts

# Sum the counts to get the total occurrences
total_occurrences = emoji_counts.sum()

# Print the total occurrences
print(f'Total occurrences of 🍉 emoji: {total_occurrences}')

# Filter rows where the emoji count is greater than 0
comments_with_emoji = filtered_df[emoji_counts > 0]

# Iterate over the DataFrame and print occurrences
for index, row in comments_with_emoji.iterrows():
    print(f'Index {index}: {row["self_text"]}')

# Plot a bar chart to visualize the occurrences
# Your plotting code goes here

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Assuming 'self_text' is the column containing posts/comments in your DataFrame
emojis_to_count = ['🍉', '🇵🇸']  # 🇵🇸 represents the Palestinian flag emoji

# Create a new column in the DataFrame for each emoji count
for emoji in emojis_to_count:
    emoji_counts = filtered_df['self_text'].str.count(emoji)
    filtered_df[f'{emoji}_counts'] = emoji_counts

# Sum the counts for each emoji to get the total occurrences
total_occurrences = filtered_df[[f'{emoji}_counts' for emoji in emojis_to_count]].sum()

# Display the total occurrences of each emoji
print(f'Total occurrences of each emoji: {total_occurrences}')

# Plot a bar chart to visualize the occurrences
plt.figure(figsize=(10, 6))
sns.barplot(x=emojis_to_count, y=total_occurrences.values, palette='viridis')
plt.xlabel('Emoji')
plt.ylabel('Total Occurrences')
plt.title('Total Occurrences of Emojis in Posts/Comments')
plt.show()
for emoji in emojis_to_count:
    emoji_counts = filtered_df['self_text'].str.count(emoji)
    filtered_df[f'{emoji}_counts'] = emoji_counts

# Sum the counts for each emoji to get the total occurrences
total_occurrences = filtered_df[[f'{emoji}_counts' for emoji in emojis_to_count]].sum()


# Plot a pie chart to visualize the occurrences
plt.figure(figsize=(8, 8))
plt.pie(total_occurrences, labels=emojis_to_count, autopct='%1.1f%%', startangle=140, colors=['lightblue', 'lightgreen'])
plt.title('Percentage of Occurrences of Emojis in Posts/Comments')
plt.show()

# **geopolitical determination**

Identify Author Locations
Reddit, a predominantly anonymous community, faces challenges in discerning the geographical locations of its users. While Reddit has access to users' IP addresses, this information remains undisclosed to the public.

An effective strategy involves recognizing that user activity, particularly engagement through comments, tends to peak from 12:00 to 21:00 UTC. By leveraging this temporal pattern, we can make a tentative inference about a user's likely country or region based on their prevalent posting times.

To achieve this, we will follow these steps:

1. Extract the hour component from the `created_time` column.
2. Aggregate the data by `author_name` and compute the mean hour for each user.
3. Utilize the mean hour to estimate the probable time zone associated with each user.
4. Summarize the count of users within each estimated time zone.

In [ ]:
# Extract hour from 'created_time'
filtered_df['hour'] = filtered_df['created_time'].dt.hour

# Group by 'author_name' and calculate the mean hour
df_users = filtered_df.groupby('author_name')['hour'].mean().reset_index()

# Function to estimate timezone from mean hour
def estimate_timezone(hour):
    '''
    estimate_timezone function takes the mean hour as input and estimates the timezone based on the assumption that the most active time
    is between 12:00 - 21:00 local time. The function then calculates the difference (local_hour) between the mean hour and 12,and constructs
    a string in the format "UTC{local_hour:+03d}:00". This string represents the estimated timezone with a UTC offset.

    For example:
    If the mean hour is 15, then local_hour would be 3, and the returned timezone would be "UTC+03:00".
    If the mean hour is 9, then local_hour would be -3, and the returned timezone would be "UTC-03:00".
    This way, you're expressing the timezone as a UTC offset relative to UTC time.
    '''
    # Assuming most active time is between 12:00 - 21:00 local time
    local_hour = int (hour - 12)
    return f"UTC{local_hour:+03d}:00"

# Apply the timezone estimation function
df_users['estimated_timezone'] = df_users['hour'].apply(estimate_timezone)

# Count the number of users in each estimated timezone
timezone_distribution = df_users['estimated_timezone'].value_counts()
timezone_distribution

In [ ]:
#Aggregate the data by author_name and compute the mean hour for each user.
filtered_df.groupby('author_name')['hour'].mean().reset_index()

In [ ]:
# Get a list of time zones
time_zones = pytz.all_timezones

# Create a mapping of UTC offsets to countries
utc_region_mapping = defaultdict(list)

for time_zone in time_zones:
    tz = pytz.timezone(time_zone)
    now = tz.localize(datetime.datetime.now())
    offset = now.utcoffset()
    utc_offset = offset.total_seconds() // 3600
    utc_region_mapping[f'{utc_offset:+03.0f}:00'].append(time_zone)

# Convert defaultdict to a regular dictionary
utc_region_mapping = dict(utc_region_mapping)

# Function to estimate region from UTC offset
def estimate_region(estimated_timezone):
    # Extract UTC offset from the 'estimated_timezone' column
    utc_offset = estimated_timezone[-6:]

    # Map UTC offset to region using the provided utc_region_mapping
    return utc_region_mapping.get(utc_offset, 'Unknown')

# Apply the region estimation function
df_users['region'] = df_users['estimated_timezone'].apply(estimate_region)

df_users.head()

# **Controversiality Analysis**

In [ ]:
filtered_df['controversiality'] = df_original['controversiality']
# Create comment_df with non-NaN values in both self_text and post_self_text
comment_df = filtered_df.dropna(subset=['self_text','post_self_text'])
# drop duplicate comments (like comments by Reddit Commuinty)
comment_df = comment_df.drop_duplicates(subset=['self_text'])
controversiality_counts = comment_df['controversiality'].value_counts()
controversiality_counts

In [ ]:
# Calculate controversiality percentages for each subreddit
controversiality_percentages = comment_df.groupby('subreddit')['controversiality'].mean().sort_values(ascending=False)

# Sort subreddits based on the highest comment_id
subreddit_sorted = comment_df.groupby('subreddit')['comment_id'].count().sort_values(ascending=False).index

# Set up subplots
num_subplots = min(12, len(controversiality_percentages))  # Display only 12 charts
num_cols = 6  # Display 6 charts per line
num_rows = (num_subplots + num_cols - 1) // num_cols
fig, axes = plt.subplots(nrows=num_rows, ncols=num_cols, figsize=(15, 5 * num_rows))
axes = axes.flatten()

# Plot pie charts for each subreddit
for i, subreddit in enumerate(subreddit_sorted[:num_subplots]):
    # Filter data for the current subreddit
    subreddit_data = comment_df[comment_df['subreddit'] == subreddit]
    # Calculate controversiality distribution
    controversiality_counts = subreddit_data['controversiality'].value_counts()
    # Plot the pie chart for the current subreddit
    axes[i].pie(controversiality_counts, labels=controversiality_counts.index, autopct='%1.1f%%', startangle=90, colors=['lightcoral', 'lightblue'])
    axes[i].set_title(f'{subreddit}')

plt.tight_layout()
plt.show()

In [ ]:
comment_df[comment_df['controversiality'] == 1].sort_values(by=['score','user_total_karma'], ascending=False).head()[['self_text', 'score', 'user_total_karma', 'controversiality']]

In [ ]:
comment_df[comment_df['controversiality'] == 0].sort_values(by=['score','user_total_karma'], ascending=False).head()[['self_text', 'score', 'user_total_karma', 'controversiality']]

# **Topic modeling**

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation

vectorizer = TfidfVectorizer(max_features=5000, stop_words='english')
df['self_text'].fillna("", inplace=True)
tfidf_matrix = vectorizer.fit_transform(df['self_text'])
lda_model = LatentDirichletAllocation(n_components=10, random_state=42, n_jobs=-1)  # Assuming 10 topics
lda_topic_matrix = lda_model.fit_transform(tfidf_matrix)
# Get the feature names (words) from the vectorizer
feature_names = vectorizer.get_feature_names_out()

# Get the top words for each topic
# Number of words for each topic
no_words_per_topic = 10  
top_words_per_topic = []
for topic_idx, topic in enumerate(lda_model.components_):
    top_words_idx = topic.argsort()[:-no_words_per_topic - 1:-1]
    top_words = [feature_names[i] for i in top_words_idx]
    top_words_per_topic.append(top_words)

# Print the topics and related words
for topic_idx, top_words in enumerate(top_words_per_topic):
    print(f"Topic {topic_idx + 1}:")
    print(", ".join(top_words))
    print()

In [ ]:
# Get the vocabulary indices for 'israel' and 'palestine' in your vectorizer
israel_word_index = vectorizer.vocabulary_['israel']
palestine_word_index = vectorizer.vocabulary_['palestine']

In [ ]:
topic_term_distribution = lda_model.components_
israel_topic_index = topic_term_distribution[:, israel_word_index].argmax()
palestine_topic_index = topic_term_distribution[:, palestine_word_index].argmax()

In [ ]:
print(f"The topic related with 'israel' is Topic {israel_topic_index + 1}:")
print(", ".join(top_words_per_topic[israel_topic_index]))
print()

# Print the topics related with 'palestine'
print(f"The topic related with 'palestine' is Topic {palestine_topic_index + 1}:")
print(", ".join(top_words_per_topic[palestine_topic_index]))
print()